# CORTEX — Stage II: Discrete Beam Selection

> **Focal Loss · Knowledge Distillation · Causal Transformer · Angle Refinement · IEEE Paper Plots**

This notebook trains **CORTEX Stage II**, which selects optimal TX/RX beam patterns (from a 255×255 codebook) for THz UAV communications.  
It operates on raw mobility traces, runs an exhaustive search teacher to generate soft labels, and trains a causal transformer student with knowledge distillation.

### Architecture
```
Input state sequence  [B, T, F=17]
      │
      ▼
Conv Front-End    (2× Conv1d + GELU)
      │
      ▼
Causal Transformer   (N× RoPE self-attention + FFN)   → context encoding
      │
   last token + mean pool → query
      │
      ├─→ Cross-attention over TX beam codebook → TX classifier  (255-way)
      ├─→ Cross-attention over RX beam codebook → RX classifier  (255-way)
      └─→ Angle Refiner MLP                    → 6-D unit vector (TX + RX)
```

**Training signal**: Focal loss on hard labels + KL-divergence distillation from exhaustive-search teacher (top-K soft targets, temperature-annealed).


In [ ]:
!pip install torch numpy matplotlib pandas scipy scikit-learn einops -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from scipy.interpolate import RegularGridInterpolator
import time, os, glob, math, json
from pathlib import Path
from sklearn.model_selection import train_test_split

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

## 1 · Setup: Output Directories

In [ ]:
# ── Option A: Google Colab + Drive ──────────────────────────────────────────
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=False)
    EXPERIMENT_ROOT = "/content/drive/MyDrive/CORTEX_experiments_255"
    print("✅ Google Drive mounted")
except ImportError:
    # ── Option B: Local ──────────────────────────────────────────────────────
    EXPERIMENT_ROOT = "outputs"
    print("ℹ️  Running locally — outputs go to ./outputs/")

CHECKPOINT_DIR = os.path.join(EXPERIMENT_ROOT, "checkpoints")
DATA_CACHE_DIR = os.path.join(EXPERIMENT_ROOT, "data_cache")
RESULTS_DIR    = os.path.join(EXPERIMENT_ROOT, "results")
LOGS_DIR       = os.path.join(EXPERIMENT_ROOT, "logs")

for d in [CHECKPOINT_DIR, DATA_CACHE_DIR, RESULTS_DIR, LOGS_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"Checkpoints : {CHECKPOINT_DIR}")
print(f"Data cache  : {DATA_CACHE_DIR}")
print(f"Results     : {RESULTS_DIR}")
print(f"Logs        : {LOGS_DIR}")

## 2 · Configuration

In [ ]:
class Config:
    # ── Beam codebook ──────────────────────────────────────────────────────
    NUM_TX_PATTERNS = 255
    NUM_RX_PATTERNS = 255

    # ── THz channel ────────────────────────────────────────────────────────
    FREQUENCY  = 272.5e9
    BANDWIDTH  = 14e9
    C          = 3e8
    KB         = 1.380649e-23
    T0         = 296
    T_SYS      = 1000
    P_TX       = 0.1

    # ── Input / sequence ──────────────────────────────────────────────────
    STATE_FEATURE_DIM = 17
    SEQUENCE_LENGTH   = 7

    # ── Model ─────────────────────────────────────────────────────────────
    D_MODEL        = 256
    N_HEADS        = 8
    N_LAYERS       = 5
    DIM_FEEDFORWARD= 512
    DROPOUT        = 0.15
    BEAM_EMBED_DIM = 128
    CONV_CHANNELS  = 128
    CONV_KERNEL    = 5
    CONV_STRIDE    = 2
    CONV_LAYERS    = 2

    # ── Training ──────────────────────────────────────────────────────────
    LEARNING_RATE = 2e-4
    BATCH_SIZE    = 16
    NUM_EPOCHS    = 250
    TRAIN_SPLIT   = 0.8

    # ── Knowledge distillation ────────────────────────────────────────────
    USE_DISTILLATION        = True
    DISTILL_TEMPERATURE_MAX = 8.0
    DISTILL_TEMPERATURE_MIN = 3.0
    DISTILL_ALPHA_MIN       = 0.2
    DISTILL_ALPHA_MAX       = 0.6
    TOPK_TX                 = 6
    TOPK_RX                 = 6

    # ── Losses ────────────────────────────────────────────────────────────
    FOCAL_GAMMA     = 2.0
    LABEL_SMOOTHING = 0.1

    # ── Angle refinement ──────────────────────────────────────────────────
    USE_ANGLE_REFINEMENT = True
    ANGLE_LOSS_WEIGHT    = 0.20
    WC_RHO               = 0.9

    # ── Checkpointing ─────────────────────────────────────────────────────
    CHECKPOINT_EVERY_N_EPOCHS  = 10
    KEEP_LAST_N_CHECKPOINTS    = 5
    DATA_GEN_CHECKPOINT_EVERY  = 50

config = Config()
print("Config ready.")

## 3 · Load Radiation Patterns & Mobility Data

In [ ]:
# ── Set paths ────────────────────────────────────────────────────────────────
# Update these to point to your data locations.
PATTERNS_DIR   = "data/255_dataset"        # folder of rows_*.csv antenna patterns
MOBILITY_FILE  = "data/mobility.csv"       # mobility trace CSV

# Auto-detect Drive locations if running on Colab
if not os.path.exists(PATTERNS_DIR):
    for candidate in [
        "/content/drive/MyDrive/2D Radiation Patterns/255_dataset",
        "/content/drive/My Drive/2D Radiation Patterns/255_dataset",
    ]:
        if os.path.exists(candidate):
            PATTERNS_DIR = candidate
            print(f"Auto-detected patterns: {PATTERNS_DIR}")
            break

if not os.path.exists(MOBILITY_FILE):
    for candidate in glob.glob("/content/drive/**/*.csv", recursive=True):
        if "mobility" in os.path.basename(candidate).lower():
            MOBILITY_FILE = candidate
            print(f"Auto-detected mobility: {MOBILITY_FILE}")
            break

In [ ]:
def load_hfss_patterns_from_csv(directory, glob_pattern="rows_*.csv", expected_num=255):
    """Load 2-D antenna radiation patterns from CSV files."""
    print(f"Loading {expected_num} radiation patterns from {directory}...")
    csv_files = sorted(glob.glob(os.path.join(directory, glob_pattern)))
    patterns = {}

    for pid, csv_file in enumerate(csv_files[:expected_num]):
        df = pd.read_csv(csv_file)

        phi_col   = next(c for c in df.columns if "phi"   in c.lower())
        theta_col = next(c for c in df.columns if "theta" in c.lower())
        gain_col  = next(c for c in df.columns if "gain"  in c.lower())

        phi_deg   = pd.to_numeric(df[phi_col], errors="coerce").values % 360
        theta_deg = pd.to_numeric(
            df[theta_col].astype(str).str.replace("deg", ""), errors="coerce"
        ).values
        gain_db   = pd.to_numeric(df[gain_col], errors="coerce").values

        phi_u   = np.sort(np.unique(phi_deg))
        theta_u = np.sort(np.unique(theta_deg))
        gain_2d = np.full((len(phi_u), len(theta_u)), -100.0)

        for i in range(len(df)):
            pi = np.argmin(np.abs(phi_u   - phi_deg[i]))
            ti = np.argmin(np.abs(theta_u - theta_deg[i]))
            gain_2d[pi, ti] = gain_db[i]

        patterns[pid] = {
            "phi_angles":   phi_u,
            "theta_angles": theta_u,
            "gain_linear":  10.0 ** (gain_2d / 10.0),
        }
        if (pid + 1) % 50 == 0:
            print(f"  {pid+1}/{expected_num}")

    print(f"✅ Loaded {len(patterns)} patterns")
    return patterns


gain_database = load_hfss_patterns_from_csv(PATTERNS_DIR)

In [ ]:
def load_mobility_trace(filepath):
    """Load a single mobility trace CSV."""
    required = ["time", "x_tx", "y_tx", "z_tx", "x_rx", "y_rx", "z_rx",
                "vx_tx", "vy_tx", "vz_tx", "vx_rx", "vy_rx", "vz_rx"]
    df = pd.read_csv(filepath)
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    print(f"✅ Loaded mobility trace: {len(df)} timesteps "
          f"({df['time'].min():.1f}s – {df['time'].max():.1f}s)")
    return df


mobility_trace = load_mobility_trace(MOBILITY_FILE)
pattern_name   = Path(MOBILITY_FILE).stem   # used as key for caching/checkpoints

## 4 · THz Channel & Exhaustive-Search Teacher

In [ ]:
class THzChannel:
    """THz channel model: path loss + molecular absorption + thermal noise."""

    def __init__(self, cfg):
        self.f, self.B, self.c = cfg.FREQUENCY, cfg.BANDWIDTH, cfg.C
        self.kB, self.T0, self.Tsys = cfg.KB, cfg.T0, cfg.T_SYS
        self.K_abs = 0.05   # molecular absorption coefficient (m⁻¹)

    def distance(self, xTx, yTx, zTx, xRx, yRx, zRx):
        return np.sqrt((xRx-xTx)**2 + (yRx-yTx)**2 + (zRx-zTx)**2)

    def angles(self, xTx, yTx, zTx, xRx, yRx, zRx):
        dx, dy, dz = xRx-xTx, yRx-yTx, zRx-zTx
        d = np.sqrt(dx**2 + dy**2 + dz**2) + 1e-10
        phi_Tx   = np.degrees(np.arctan2(dy, dx)) % 360
        theta_Tx = np.degrees(np.arccos(np.clip(dz/d, -1, 1)))
        phi_Rx   = (phi_Tx + 180) % 360
        theta_Rx = 180 - theta_Tx
        return phi_Tx, theta_Tx, phi_Rx, theta_Rx

    def channel_gain(self, d):
        h_s  = self.c / (4 * np.pi * self.f * d)
        tau  = np.exp(-self.K_abs * d)
        return (h_s * np.sqrt(tau))**2, tau

    def noise(self, tau):
        return self.kB * (self.Tsys + self.T0 * (1 - tau)) * self.B

    def capacity(self, GTx, GRx, ch_gain_sq, P_noise):
        SNR = (config.P_TX * GTx * GRx * ch_gain_sq) / P_noise
        return self.B * np.log2(1 + SNR), SNR


channel = THzChannel(config)


class ExhaustiveSearchTeacher:
    """
    Brute-force beam selector (255×255).
    Provides hard labels AND per-beam capacity scores (soft targets).
    """

    def __init__(self, cfg, gain_db, ch):
        self.cfg = cfg
        self.ch  = ch
        self.interp = {}
        for pid, pdata in gain_db.items():
            self.interp[pid] = RegularGridInterpolator(
                (pdata["phi_angles"], pdata["theta_angles"]),
                pdata["gain_linear"],
                bounds_error=False, fill_value=None, method="linear",
            )

    def gain(self, pid, phi, theta):
        return max(self.interp[pid]([phi % 360, np.clip(theta, 0, 180)])[0], 1e-10)

    def select(self, xTx, yTx, zTx, xRx, yRx, zRx):
        """Return (best_tx_idx, best_rx_idx, best_capacity)."""
        d  = self.ch.distance(xTx, yTx, zTx, xRx, yRx, zRx)
        ph_t, th_t, ph_r, th_r = self.ch.angles(xTx, yTx, zTx, xRx, yRx, zRx)
        cg, tau = self.ch.channel_gain(d)
        Pn = self.ch.noise(tau)

        best_cap, best_tx, best_rx = -np.inf, 0, 0
        for iTx in range(self.cfg.NUM_TX_PATTERNS):
            for iRx in range(self.cfg.NUM_RX_PATTERNS):
                cap, _ = self.ch.capacity(
                    self.gain(iTx, ph_t, th_t),
                    self.gain(iRx, ph_r, th_r), cg, Pn)
                if cap > best_cap:
                    best_cap, best_tx, best_rx = cap, iTx, iRx
        return best_tx, best_rx, best_cap

    def beam_scores(self, xTx, yTx, zTx, xRx, yRx, zRx):
        """Per-beam capacity scores for soft-target distillation."""
        d  = self.ch.distance(xTx, yTx, zTx, xRx, yRx, zRx)
        ph_t, th_t, ph_r, th_r = self.ch.angles(xTx, yTx, zTx, xRx, yRx, zRx)
        cg, tau = self.ch.channel_gain(d)
        Pn = self.ch.noise(tau)
        G0 = self.gain(0, ph_r, th_r)

        tx_scores = np.array([
            self.ch.capacity(self.gain(i, ph_t, th_t), G0, cg, Pn)[0]
            for i in range(self.cfg.NUM_TX_PATTERNS)
        ])
        G0tx = self.gain(0, ph_t, th_t)
        rx_scores = np.array([
            self.ch.capacity(G0tx, self.gain(i, ph_r, th_r), cg, Pn)[0]
            for i in range(self.cfg.NUM_RX_PATTERNS)
        ])
        return tx_scores, rx_scores


teacher = ExhaustiveSearchTeacher(config, gain_database, channel)
print("✅ Channel and exhaustive-search teacher ready")

## 5 · Checkpoint & Cache Managers

In [ ]:
class CheckpointManager:
    """Save / load model training checkpoints, keeping the last N."""

    def __init__(self, checkpoint_dir, pattern_name):
        self.ckpt_dir = Path(checkpoint_dir)
        self.ckpt_dir.mkdir(parents=True, exist_ok=True)
        self.prefix = "ckpt_" + pattern_name.replace("/", "_").replace(" ", "_")[:100]

    def save(self, epoch, model, optimizer, scheduler, metrics, is_best=False):
        state = {
            "epoch": epoch,
            "model_state_dict":     model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "metrics":  metrics,
            "timestamp": time.time(),
        }
        path = self.ckpt_dir / f"{self.prefix}_epoch_{epoch:03d}.pth"
        torch.save(state, path)
        torch.save(state, self.ckpt_dir / f"{self.prefix}_latest.pth")
        if is_best:
            torch.save(state, self.ckpt_dir / f"{self.prefix}_best.pth")
            print(f"  🌟 Best checkpoint saved (epoch {epoch})")

        # Prune old checkpoints
        ckpts = sorted(self.ckpt_dir.glob(f"{self.prefix}_epoch_*.pth"),
                       key=lambda p: p.stat().st_mtime)
        for old in ckpts[:-config.KEEP_LAST_N_CHECKPOINTS]:
            old.unlink()

    def load_latest(self):
        path = self.ckpt_dir / f"{self.prefix}_latest.pth"
        if path.exists():
            return torch.load(path, map_location=device)
        ckpts = sorted(self.ckpt_dir.glob(f"{self.prefix}_epoch_*.pth"),
                       key=lambda p: p.stat().st_mtime)
        if ckpts:
            return torch.load(ckpts[-1], map_location=device)
        return None

    def load_best(self):
        path = self.ckpt_dir / f"{self.prefix}_best.pth"
        if path.exists():
            return torch.load(path, map_location=device)
        return None

    def exists(self):
        return (self.ckpt_dir / f"{self.prefix}_latest.pth").exists()


class DataCacheManager:
    """Cache expensive exhaustive-search training data generation."""

    def __init__(self, cache_dir):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(parents=True, exist_ok=True)

    def _path(self, name, partial=False):
        safe = name.replace("/", "_").replace(" ", "_")[:100]
        suffix = "_partial.npz" if partial else ".npz"
        return self.cache_dir / f"data_{safe}{suffix}"

    def save(self, name, sequences, labels_Tx, labels_Rx,
             soft_tx, soft_rx, angle_vecs, partial_idx=None):
        path = self._path(name, partial=partial_idx is not None)
        kwargs = dict(sequences=np.array(sequences, dtype=np.float32),
                      labels_Tx=np.array(labels_Tx),
                      labels_Rx=np.array(labels_Rx),
                      soft_tx=np.array(soft_tx,  dtype=np.float32),
                      soft_rx=np.array(soft_rx,  dtype=np.float32),
                      angle_vecs=np.array(angle_vecs, dtype=np.float32),
                      timestamp=time.time())
        if partial_idx is not None:
            kwargs["last_idx"] = partial_idx
        np.savez_compressed(path, **kwargs)
        if partial_idx is None:
            # Remove partial file on successful completion
            p = self._path(name, partial=True)
            if p.exists(): p.unlink()

    def load(self, name, partial=False):
        path = self._path(name, partial=partial)
        if not path.exists(): return None
        d = np.load(path)
        result = {k: d[k] for k in d.files}
        if partial:
            result["sequences"]  = result["sequences"].tolist()
            result["labels_Tx"]  = result["labels_Tx"].tolist()
            result["labels_Rx"]  = result["labels_Rx"].tolist()
            result["soft_tx"]    = result["soft_tx"].tolist()
            result["soft_rx"]    = result["soft_rx"].tolist()
            result["angle_vecs"] = result["angle_vecs"].tolist()
            result["last_idx"]   = int(result["last_idx"])
        return result

    def exists(self, name): return self._path(name).exists()
    def partial_exists(self, name): return self._path(name, partial=True).exists()


ckpt_manager  = CheckpointManager(CHECKPOINT_DIR, pattern_name)
cache_manager = DataCacheManager(DATA_CACHE_DIR)
print("✅ Checkpoint and cache managers ready")

## 6 · Angle Utilities

In [ ]:
def angles_to_unit_vector(phi_Tx, theta_Tx, phi_Rx, theta_Rx):
    """Pack four angles (degrees) into a 6-D unit-vector array [tx(3) | rx(3)]."""
    def sph2cart(phi_d, theta_d):
        p, t = np.deg2rad(phi_d), np.deg2rad(theta_d)
        return np.array([np.sin(t)*np.cos(p), np.sin(t)*np.sin(p), np.cos(t)])
    return np.concatenate([sph2cart(phi_Tx, theta_Tx),
                           sph2cart(phi_Rx, theta_Rx)]).astype(np.float32)

## 7 · Training Data Generation (with Incremental Checkpointing)

In [ ]:
def generate_training_data(trace, teacher, channel, seq_len,
                            name, cache_mgr):
    """
    Run exhaustive search over every timestep to produce:
      - sequences     : [N, T, 17]  state feature windows
      - labels_Tx/Rx  : [N]         hard beam indices
      - soft_tx/rx    : [N, 255]    per-beam capacity scores (for KD)
      - angle_vecs    : [N, 6]      unit vectors for angle-refinement loss

    Results are cached to disk. Partial checkpoints are saved every
    config.DATA_GEN_CHECKPOINT_EVERY samples so generation can resume
    after an interruption.
    """
    # ── Try full cache ────────────────────────────────────────────────────
    cached = cache_mgr.load(name)
    if cached is not None:
        print("✅ Loaded data from cache")
        return (cached["sequences"], cached["labels_Tx"], cached["labels_Rx"],
                cached["soft_tx"],   cached["soft_rx"],   cached["angle_vecs"])

    # ── Try partial checkpoint ────────────────────────────────────────────
    partial = cache_mgr.load(name, partial=True)
    if partial:
        seqs, lTx, lRx = partial["sequences"], partial["labels_Tx"], partial["labels_Rx"]
        stx, srx, avecs = partial["soft_tx"], partial["soft_rx"], partial["angle_vecs"]
        start = partial["last_idx"] + 1
        print(f"🔄 Resuming from index {start} ({len(seqs)} samples already done)")
    else:
        seqs, lTx, lRx, stx, srx, avecs = [], [], [], [], [], []
        start = seq_len - 1

    total   = len(trace) - (seq_len - 1)
    freq    = config.DATA_GEN_CHECKPOINT_EVERY
    t_start = time.time()
    since_ckpt = 0

    print(f"Generating {total} samples (checkpointing every {freq})…")

    for idx in range(start, len(trace)):
        # ── Build feature window ──────────────────────────────────────────
        window = []
        for t in range(idx - seq_len + 1, idx + 1):
            r = trace.iloc[t]
            xT, yT, zT = r["x_tx"], r["y_tx"], r["z_tx"]
            xR, yR, zR = r["x_rx"], r["y_rx"], r["z_rx"]
            d   = channel.distance(xT, yT, zT, xR, yR, zR)
            phT, thT, _, _ = channel.angles(xT, yT, zT, xR, yR, zR)
            window.append([
                xT/100, yT/100, zT/100, xR/100, yR/100, zR/100, d/100,
                r["vx_tx"]/10, r["vy_tx"]/10, r["vz_tx"]/10,
                r["vx_rx"]/10, r["vy_rx"]/10, r["vz_rx"]/10,
                np.sin(np.deg2rad(phT)), np.cos(np.deg2rad(phT)),
                np.sin(np.deg2rad(thT)), np.cos(np.deg2rad(thT)),
            ])

        # ── Exhaustive search ─────────────────────────────────────────────
        r = trace.iloc[idx]
        xT, yT, zT = r["x_tx"], r["y_tx"], r["z_tx"]
        xR, yR, zR = r["x_rx"], r["y_rx"], r["z_rx"]

        bTx, bRx, _ = teacher.select(xT, yT, zT, xR, yR, zR)
        sc_tx, sc_rx = teacher.beam_scores(xT, yT, zT, xR, yR, zR)
        phT, thT, phR, thR = channel.angles(xT, yT, zT, xR, yR, zR)

        seqs.append(window)
        lTx.append(bTx);  lRx.append(bRx)
        stx.append(sc_tx.astype(np.float32))
        srx.append(sc_rx.astype(np.float32))
        avecs.append(angles_to_unit_vector(phT, thT, phR, thR))

        since_ckpt += 1
        n_done = len(seqs)

        if since_ckpt >= freq:
            cache_mgr.save(name, seqs, lTx, lRx, stx, srx, avecs, partial_idx=idx)
            elapsed = time.time() - t_start
            rate    = n_done / elapsed
            eta_min = (total - n_done) / rate / 60 if rate > 0 else 0
            print(f"  💾 {n_done}/{total} ({100*n_done/total:.1f}%)  "
                  f"rate={rate:.2f}/s  ETA={eta_min:.1f} min")
            since_ckpt = 0
        elif n_done % 50 == 0:
            elapsed = time.time() - t_start
            rate    = n_done / elapsed if elapsed > 0 else 0
            eta_min = (total - n_done) / rate / 60 if rate > 0 else 0
            print(f"  📊 {n_done}/{total} ({100*n_done/total:.1f}%)  "
                  f"rate={rate:.2f}/s  ETA={eta_min:.1f} min")

    seqs    = np.array(seqs,    dtype=np.float32)
    lTx     = np.array(lTx)
    lRx     = np.array(lRx)
    stx     = np.array(stx,    dtype=np.float32)
    srx     = np.array(srx,    dtype=np.float32)
    avecs   = np.array(avecs,  dtype=np.float32)

    total_t = time.time() - t_start
    print(f"\n✅ Generated {len(seqs)} samples in {total_t/60:.1f} min")
    cache_mgr.save(name, seqs, lTx, lRx, stx, srx, avecs)
    return seqs, lTx, lRx, stx, srx, avecs


sequences, labels_Tx, labels_Rx, soft_tx, soft_rx, angle_vecs = \
    generate_training_data(mobility_trace, teacher, channel,
                           config.SEQUENCE_LENGTH, pattern_name, cache_manager)

print(f"Dataset shape : {sequences.shape}")

## 8 · Model — Causal Transformer with RoPE & Beam Cross-Attention

In [ ]:
# ── RoPE utilities ───────────────────────────────────────────────────────────

def rotate_half(x):
    x1, x2 = x[..., :x.shape[-1]//2], x[..., x.shape[-1]//2:]
    return torch.cat((-x2, x1), dim=-1)

class RotaryEmbedding(nn.Module):
    def __init__(self, dim, base=10000):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)

    def forward(self, seq_len, device):
        t     = torch.arange(seq_len, device=device).type_as(self.inv_freq)
        freqs = torch.einsum("t,f->tf", t, self.inv_freq)
        emb   = torch.cat([freqs, freqs], dim=-1)
        return emb.cos()[:, None, :], emb.sin()[:, None, :]

def apply_rope(q, k, cos, sin):
    return (q * cos) + (rotate_half(q) * sin), (k * cos) + (rotate_half(k) * sin)


# ── Attention block ───────────────────────────────────────────────────────────

class CausalRoPEAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.0):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads  = n_heads
        self.head_dim = d_model // n_heads
        self.q = nn.Linear(d_model, d_model)
        self.k = nn.Linear(d_model, d_model)
        self.v = nn.Linear(d_model, d_model)
        self.o = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.rope  = RotaryEmbedding(self.head_dim)

    def forward(self, x):
        B, T, D = x.shape
        H = self.n_heads
        q = self.q(x).view(B, T, H, self.head_dim)
        k = self.k(x).view(B, T, H, self.head_dim)
        v = self.v(x).view(B, T, H, self.head_dim)
        cos, sin = self.rope(T, x.device)
        q, k = apply_rope(q, k, cos, sin)
        att  = torch.einsum("bthd,bshd->bhts", q, k) / math.sqrt(self.head_dim)
        mask = torch.ones(T, T, device=x.device).tril()
        att  = att.masked_fill(mask == 0, float("-inf"))
        att  = self.drop(torch.softmax(att, dim=-1))
        out  = torch.einsum("bhts,bshd->bthd", att, v).contiguous().view(B, T, D)
        return self.o(out)

class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, dim_ff, dropout):
        super().__init__()
        self.ln1  = nn.LayerNorm(d_model)
        self.attn = CausalRoPEAttention(d_model, n_heads, dropout)
        self.drop1= nn.Dropout(dropout)
        self.ln2  = nn.LayerNorm(d_model)
        self.ff   = nn.Sequential(
            nn.Linear(d_model, dim_ff), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(dim_ff, d_model),
        )
        self.drop2= nn.Dropout(dropout)

    def forward(self, x):
        x = x + self.drop1(self.attn(self.ln1(x)))
        x = x + self.drop2(self.ff(self.ln2(x)))
        return x


# ── Temporal encoder (Conv + Transformer) ────────────────────────────────────

class TemporalEncoder(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        C = cfg.CONV_CHANNELS
        K = cfg.CONV_KERNEL
        self.conv = nn.Sequential(
            nn.Conv1d(cfg.STATE_FEATURE_DIM, C, K, stride=cfg.CONV_STRIDE, padding=K//2),
            nn.GELU(),
            nn.Conv1d(C, C, K, stride=cfg.CONV_STRIDE, padding=K//2),
            nn.GELU(),
        )
        self.proj   = nn.Linear(C, cfg.D_MODEL)
        self.blocks = nn.ModuleList([
            TransformerBlock(cfg.D_MODEL, cfg.N_HEADS, cfg.DIM_FEEDFORWARD, cfg.DROPOUT)
            for _ in range(cfg.N_LAYERS)
        ])

    def forward(self, x):               # x: [B, T, F]
        z = self.conv(x.transpose(1,2)).transpose(1,2)
        z = self.proj(z)
        for blk in self.blocks: z = blk(z)
        return z


# ── Full model ────────────────────────────────────────────────────────────────

class CORTEXBeamSelector(nn.Module):
    """
    CORTEX Stage II.
    Input : [B, T, 17]
    Output: tx_logits [B,255], rx_logits [B,255], angle_pred [B,6],
            tx_conf [B,1], rx_conf [B,1]
    """
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg

        self.encoder = TemporalEncoder(cfg)

        # Learnable beam codebooks
        self.tx_codebook = nn.Parameter(torch.randn(cfg.NUM_TX_PATTERNS, cfg.BEAM_EMBED_DIM) * 0.02)
        self.rx_codebook = nn.Parameter(torch.randn(cfg.NUM_RX_PATTERNS, cfg.BEAM_EMBED_DIM) * 0.02)

        self.tx_kv = nn.Linear(cfg.BEAM_EMBED_DIM, cfg.D_MODEL)
        self.rx_kv = nn.Linear(cfg.BEAM_EMBED_DIM, cfg.D_MODEL)

        self.tx_cross = CausalRoPEAttention(cfg.D_MODEL, cfg.N_HEADS, cfg.DROPOUT)
        self.rx_cross = CausalRoPEAttention(cfg.D_MODEL, cfg.N_HEADS, cfg.DROPOUT)
        self.tx_res   = nn.Linear(cfg.D_MODEL, cfg.D_MODEL)
        self.rx_res   = nn.Linear(cfg.D_MODEL, cfg.D_MODEL)

        def _head(in_dim, out_dim):
            return nn.Sequential(
                nn.Linear(in_dim, in_dim),   nn.LayerNorm(in_dim),   nn.GELU(), nn.Dropout(cfg.DROPOUT),
                nn.Linear(in_dim, in_dim//2), nn.LayerNorm(in_dim//2), nn.GELU(), nn.Dropout(cfg.DROPOUT),
                nn.Linear(in_dim//2, out_dim),
            )

        self.tx_cls  = _head(cfg.D_MODEL, cfg.NUM_TX_PATTERNS)
        self.rx_cls  = _head(cfg.D_MODEL, cfg.NUM_RX_PATTERNS)

        self.tx_conf = nn.Sequential(nn.Linear(cfg.D_MODEL, cfg.D_MODEL//4),
                                     nn.ReLU(), nn.Linear(cfg.D_MODEL//4, 1), nn.Sigmoid())
        self.rx_conf = nn.Sequential(nn.Linear(cfg.D_MODEL, cfg.D_MODEL//4),
                                     nn.ReLU(), nn.Linear(cfg.D_MODEL//4, 1), nn.Sigmoid())

        if cfg.USE_ANGLE_REFINEMENT:
            self.angle_refiner = nn.Sequential(
                nn.Linear(cfg.D_MODEL*2, cfg.D_MODEL), nn.LayerNorm(cfg.D_MODEL),
                nn.GELU(), nn.Dropout(cfg.DROPOUT),
                nn.Linear(cfg.D_MODEL, cfg.D_MODEL//2), nn.LayerNorm(cfg.D_MODEL//2),
                nn.GELU(), nn.Linear(cfg.D_MODEL//2, 6),
            )

    def forward(self, x):
        B  = x.size(0)
        enc= self.encoder(x)                                    # [B, T', D]
        q  = enc[:, -1:, :] + 0.5 * enc.mean(dim=1, keepdim=True)  # [B,1,D]

        def _cross(query, codebook_proj, cross_attn, residual_proj):
            kv  = codebook_proj.unsqueeze(0).expand(B, -1, -1)  # [B, num_beams, D]
            seq = torch.cat([query, kv], dim=1)
            out = cross_attn(seq)[:, 0, :]                      # [B, D]
            return query.squeeze(1) + residual_proj(out)

        tx_out = _cross(q, self.tx_kv(self.tx_codebook), self.tx_cross, self.tx_res)
        rx_out = _cross(q, self.rx_kv(self.rx_codebook), self.rx_cross, self.rx_res)

        tx_logits = self.tx_cls(tx_out)
        rx_logits = self.rx_cls(rx_out)
        tx_conf   = self.tx_conf(tx_out)
        rx_conf   = self.rx_conf(rx_out)

        angles = None
        if self.cfg.USE_ANGLE_REFINEMENT:
            angles = self.angle_refiner(torch.cat([tx_out, rx_out], dim=1))
        return tx_logits, rx_logits, angles, tx_conf, rx_conf


model = CORTEXBeamSelector(config).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"Parameters: {n_params:,}  ({n_params*4/1024/1024:.1f} MB)")

## 9 · Loss Functions

In [ ]:
class FocalLoss(nn.Module):
    """Focal loss — down-weights easy examples to focus training on hard ones."""
    def __init__(self, gamma=2.0, label_smoothing=0.0):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(label_smoothing=label_smoothing, reduction="none")

    def forward(self, logits, targets):
        ce   = self.ce(logits, targets)
        pt   = torch.exp(-ce)
        return ((1 - pt) ** self.gamma * ce).mean()


def distillation_loss(student_logits, soft_targets, temperature):
    """KL-divergence KD loss (temperature-scaled)."""
    log_p = torch.log_softmax(student_logits / temperature, dim=1)
    q     = torch.softmax(soft_targets / temperature, dim=1)
    return nn.KLDivLoss(reduction="batchmean")(log_p, q) * (temperature ** 2)


class WrappedCauchyLoss(nn.Module):
    """Circular-aware loss for 6-D unit vectors (TX + RX angle refinement)."""
    def __init__(self, rho=0.9, eps=1e-8):
        super().__init__()
        self.rho, self.eps = rho, eps

    def forward(self, pred, target):
        def norm(v): return v / (v.norm(dim=1, keepdim=True) + self.eps)
        ptx, prx = norm(pred[:,  :3]), norm(pred[:,  3:])
        ttx, trx = norm(target[:,:3]), norm(target[:,3:])
        cos_tx = (ptx * ttx).sum(1).clamp(-1+self.eps, 1-self.eps)
        cos_rx = (prx * trx).sum(1).clamp(-1+self.eps, 1-self.eps)
        rho2   = self.rho**2
        return (torch.log(1 + rho2 - 2*self.rho*cos_tx + self.eps) +
                torch.log(1 + rho2 - 2*self.rho*cos_rx + self.eps)).mean()


focal_loss  = FocalLoss(gamma=config.FOCAL_GAMMA, label_smoothing=config.LABEL_SMOOTHING)
angle_loss  = WrappedCauchyLoss(rho=config.WC_RHO)
print("Loss functions ready.")

## 10 · Dataset & DataLoaders

In [ ]:
class BeamDataset(Dataset):
    def __init__(self, seqs, lTx, lRx, stx, srx, avecs):
        self.seqs  = torch.FloatTensor(seqs)
        self.lTx   = torch.LongTensor(lTx)
        self.lRx   = torch.LongTensor(lRx)
        self.stx   = torch.FloatTensor(stx)
        self.srx   = torch.FloatTensor(srx)
        self.avecs = torch.FloatTensor(avecs)

    def __len__(self): return len(self.seqs)

    def __getitem__(self, i):
        return self.seqs[i], self.lTx[i], self.lRx[i], self.stx[i], self.srx[i], self.avecs[i]


(tr_s, va_s, tr_Tx, va_Tx, tr_Rx, va_Rx,
 tr_stx, va_stx, tr_srx, va_srx, tr_av, va_av) = train_test_split(
    sequences, labels_Tx, labels_Rx, soft_tx, soft_rx, angle_vecs,
    test_size=1-config.TRAIN_SPLIT, random_state=SEED
)

train_ds = BeamDataset(tr_s,  tr_Tx, tr_Rx,  tr_stx,  tr_srx,  tr_av)
val_ds   = BeamDataset(va_s,  va_Tx, va_Rx,  va_stx,  va_srx,  va_av)

train_loader = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=config.BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train: {len(train_ds):,}  |  Val: {len(val_ds):,}")

## 11 · Training

In [ ]:
def lin_schedule(start, end, step, total):
    return start + (end - start) * min(max(step, 0), total) / max(total, 1)

def topk_masked(logits, k):
    """Zero out all but the top-K values (for sparse KD targets)."""
    if k <= 0 or k >= logits.size(1): return logits
    _, idx = torch.topk(logits, k, dim=1)
    return torch.zeros_like(logits).scatter_(1, idx, logits.gather(1, idx))


def run_epoch(model, loader, optimizer, training, epoch):
    model.train() if training else model.eval()
    total_loss = 0
    correct_tx = correct_rx = correct_both = n = 0

    T     = lin_schedule(config.DISTILL_TEMPERATURE_MAX, config.DISTILL_TEMPERATURE_MIN,
                         epoch, config.NUM_EPOCHS - 1)
    alpha = lin_schedule(config.DISTILL_ALPHA_MIN, config.DISTILL_ALPHA_MAX,
                         epoch, config.NUM_EPOCHS - 1)

    ctx = torch.enable_grad() if training else torch.no_grad()
    with ctx:
        for seq, yTx, yRx, stx, srx, avec in loader:
            seq  = seq.to(device);  yTx  = yTx.to(device); yRx = yRx.to(device)
            stx  = stx.to(device);  srx  = srx.to(device); avec= avec.to(device)

            tx_log, rx_log, ang, _, _ = model(seq)

            l_tx_hard = focal_loss(tx_log, yTx)
            l_rx_hard = focal_loss(rx_log, yRx)

            if training and config.USE_DISTILLATION:
                l_tx = alpha * l_tx_hard + (1-alpha) * distillation_loss(tx_log, topk_masked(stx, config.TOPK_TX), T)
                l_rx = alpha * l_rx_hard + (1-alpha) * distillation_loss(rx_log, topk_masked(srx, config.TOPK_RX), T)
            else:
                l_tx, l_rx = l_tx_hard, l_rx_hard

            loss = l_tx + l_rx
            if config.USE_ANGLE_REFINEMENT and ang is not None:
                loss = loss + config.ANGLE_LOSS_WEIGHT * angle_loss(ang, avec)

            if training:
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

            total_loss   += loss.item()
            pred_tx       = tx_log.argmax(1);  pred_rx = rx_log.argmax(1)
            correct_tx   += (pred_tx == yTx).sum().item()
            correct_rx   += (pred_rx == yRx).sum().item()
            correct_both += ((pred_tx == yTx) & (pred_rx == yRx)).sum().item()
            n            += yTx.size(0)

    return total_loss/len(loader), correct_tx/n, correct_rx/n, correct_both/n


optimizer = optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=2, eta_min=1e-7)

# ── Resume from checkpoint if available ──────────────────────────────────────
start_epoch = 0
best_val_acc = 0.0

prev_ckpt = ckpt_manager.load_latest()
if prev_ckpt is not None:
    model.load_state_dict(prev_ckpt["model_state_dict"])
    optimizer.load_state_dict(prev_ckpt["optimizer_state_dict"])
    scheduler.load_state_dict(prev_ckpt["scheduler_state_dict"])
    start_epoch  = prev_ckpt["epoch"] + 1
    best_val_acc = prev_ckpt["metrics"]["best_val_acc"]
    print(f"🔄 Resumed from epoch {start_epoch}  (best val: {best_val_acc*100:.1f}%)")
else:
    print("Starting fresh training.")

print(f"Training for {config.NUM_EPOCHS - start_epoch} more epochs…")

In [ ]:
log_path = os.path.join(LOGS_DIR, f"log_{pattern_name[:80]}.csv")
if start_epoch == 0:
    with open(log_path, "w") as f:
        f.write("epoch,train_loss,val_loss,train_both,val_both,lr,time\n")

for epoch in range(start_epoch, config.NUM_EPOCHS):
    t0 = time.time()
    tr_loss, tr_tx, tr_rx, tr_both = run_epoch(model, train_loader, optimizer, True,  epoch)
    va_loss, va_tx, va_rx, va_both = run_epoch(model, val_loader,   optimizer, False, epoch)
    scheduler.step()

    lr      = optimizer.param_groups[0]["lr"]
    elapsed = time.time() - t0
    is_best = va_both > best_val_acc
    if is_best: best_val_acc = va_both

    with open(log_path, "a") as f:
        f.write(f"{epoch},{tr_loss:.6f},{va_loss:.6f},{tr_both:.6f},{va_both:.6f},{lr:.8f},{elapsed:.1f}\n")

    save_now = is_best or (epoch+1) % config.CHECKPOINT_EVERY_N_EPOCHS == 0 or epoch == config.NUM_EPOCHS-1
    if save_now:
        ckpt_manager.save(epoch, model, optimizer, scheduler,
                         {"train_loss": tr_loss, "val_loss": va_loss,
                          "train_both_acc": tr_both, "val_both_acc": va_both,
                          "best_val_acc": best_val_acc},
                         is_best=is_best)

    if epoch % 20 == 0 or epoch == config.NUM_EPOCHS - 1:
        print(f"Epoch {epoch:3d}: "
              f"TrLoss={tr_loss:.4f}  ValLoss={va_loss:.4f}  "
              f"Tx={va_tx*100:.1f}%  Rx={va_rx*100:.1f}%  Both={va_both*100:.1f}% "
              f"{'🌟' if is_best else '  '}  LR={lr:.1e}  {elapsed:.1f}s")

print(f"\n✅ Training complete. Best joint acc: {best_val_acc*100:.1f}%")

## 12 · Evaluation — Spectral Efficiency vs. Exhaustive Search

In [ ]:
# Load best checkpoint
best_ckpt = ckpt_manager.load_best()
if best_ckpt:
    model.load_state_dict(best_ckpt["model_state_dict"])
    print(f"Best model loaded (epoch {best_ckpt['epoch']}, "
          f"val acc {best_ckpt['metrics']['best_val_acc']*100:.1f}%)")
model.eval()


def evaluate_spectral_efficiency(model, trace, teacher, channel, cfg,
                                 name, results_dir):
    """
    Compare CORTEX vs. exhaustive search on every timestep.
    Saves incremental CSV checkpoints every 100 steps.
    Returns a DataFrame with capacity columns.
    """
    safe      = name.replace("/","_").replace(" ","_")[:80]
    ckpt_csv  = os.path.join(results_dir, f"eval_ckpt_{safe}.csv")
    final_csv = os.path.join(results_dir, f"results_{safe}.csv")

    # Resume if an eval checkpoint exists
    rows, start = [], cfg.SEQUENCE_LENGTH - 1
    if os.path.exists(ckpt_csv):
        df_prev = pd.read_csv(ckpt_csv)
        rows    = df_prev.to_dict("records")
        start   = len(rows) + cfg.SEQUENCE_LENGTH - 1
        print(f"🔄 Resuming eval from step {start} ({len(rows)} done)")

    for idx in range(start, len(trace)):
        window = []
        for t in range(idx - cfg.SEQUENCE_LENGTH + 1, idx + 1):
            r = trace.iloc[t]
            xT,yT,zT = r["x_tx"],r["y_tx"],r["z_tx"]
            xR,yR,zR = r["x_rx"],r["y_rx"],r["z_rx"]
            d = channel.distance(xT,yT,zT,xR,yR,zR)
            phT,thT,_,_ = channel.angles(xT,yT,zT,xR,yR,zR)
            window.append([
                xT/100,yT/100,zT/100,xR/100,yR/100,zR/100,d/100,
                r["vx_tx"]/10,r["vy_tx"]/10,r["vz_tx"]/10,
                r["vx_rx"]/10,r["vy_rx"]/10,r["vz_rx"]/10,
                np.sin(np.deg2rad(phT)),np.cos(np.deg2rad(phT)),
                np.sin(np.deg2rad(thT)),np.cos(np.deg2rad(thT)),
            ])

        with torch.no_grad():
            t_in = torch.FloatTensor([window]).to(device)
            tx_log, rx_log, _, _, _ = model(t_in)
            tx_pred = tx_log.argmax(1).item()
            rx_pred = rx_log.argmax(1).item()

        r   = trace.iloc[idx]
        xT,yT,zT = r["x_tx"],r["y_tx"],r["z_tx"]
        xR,yR,zR = r["x_rx"],r["y_rx"],r["z_rx"]
        phT,thT,phR,thR = channel.angles(xT,yT,zT,xR,yR,zR)
        d   = channel.distance(xT,yT,zT,xR,yR,zR)
        cg, tau = channel.channel_gain(d)
        Pn  = channel.noise(tau)

        GTx  = teacher.gain(tx_pred, phT, thT)
        GRx  = teacher.gain(rx_pred, phR, thR)
        cap, _  = channel.capacity(GTx, GRx, cg, Pn)
        _,_,opt = teacher.select(xT,yT,zT,xR,yR,zR)

        rows.append({
            "time":         r["time"],
            "capacity":     cap / 1e9,
            "opt_capacity": opt / 1e9,
            "tx_beam":      tx_pred,
            "rx_beam":      rx_pred,
        })

        done = idx + 1 - (cfg.SEQUENCE_LENGTH - 1)
        if done % 100 == 0:
            pd.DataFrame(rows).to_csv(ckpt_csv, index=False)
            total = len(trace) - (cfg.SEQUENCE_LENGTH - 1)
            print(f"  💾 {done}/{total} ({100*done/total:.1f}%)")

    df = pd.DataFrame(rows)
    df.to_csv(final_csv, index=False)
    if os.path.exists(ckpt_csv): os.remove(ckpt_csv)
    print(f"✅ Evaluation saved: {final_csv}")
    return df


results = evaluate_spectral_efficiency(
    model, mobility_trace, teacher, channel,
    config, pattern_name, RESULTS_DIR
)

avg_cap  = results["capacity"].mean()
avg_opt  = results["opt_capacity"].mean()
eff      = avg_cap / avg_opt
print(f"\nCORTEX     : {avg_cap:.4f} Gbps")
print(f"Exhaustive : {avg_opt:.4f} Gbps")
print(f"Efficiency : {eff*100:.1f}%")
print(f"Gap        : {(1-eff)*100:.2f}%")

## 13 · IEEE Paper Plots

In [ ]:
import matplotlib.pyplot as plt

PLOTS_DIR = os.path.join(EXPERIMENT_ROOT, "paper_plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

plt.rcParams.update({
    "font.family":      "serif",
    "font.serif":       ["Times New Roman"],
    "font.size":        10,
    "axes.labelsize":   10,
    "axes.titlesize":   10,
    "legend.fontsize":   9,
    "xtick.labelsize":   9,
    "ytick.labelsize":   9,
    "lines.linewidth":  1.5,
    "lines.markersize":  4,
    "grid.alpha":       0.3,
})

def save_fig(name):
    for ext in ("png", "pdf"):
        plt.savefig(os.path.join(PLOTS_DIR, f"{name}.{ext}"),
                    dpi=300, bbox_inches="tight", format=ext)
    print(f"  Saved: {name}.png / .pdf")

In [ ]:
# ── (a) Training dynamics ─────────────────────────────────────────────────────
df_log = pd.read_csv(log_path)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.16, 3.0))

ax1.plot(df_log["epoch"], df_log["train_loss"], label="Train")
ax1.plot(df_log["epoch"], df_log["val_loss"],   label="Validation", linestyle="--")
ax1.set(xlabel="Epoch", ylabel="Loss", title="(a) Training Convergence")
ax1.legend(); ax1.grid()

ax2.plot(df_log["epoch"], df_log["train_both"]*100, label="Train (Joint)")
ax2.plot(df_log["epoch"], df_log["val_both"]  *100, label="Val (Joint)", linestyle="--")
ax2.set(xlabel="Epoch", ylabel="Accuracy (%)", title="(b) Joint Beam Accuracy")
ax2.legend(); ax2.grid()

plt.tight_layout()
save_fig("fig_training_dynamics")
plt.show()

In [ ]:
# ── (b) Spectral efficiency CDF ───────────────────────────────────────────────
BW_GHZ = config.BANDWIDTH / 1e9
se_pred = results["capacity"]     / BW_GHZ
se_opt  = results["opt_capacity"] / BW_GHZ

fig, ax = plt.subplots(figsize=(3.5, 3.0))

for vals, label, color, ls in [
    (se_opt,  "Optimal (Exhaustive)", "k",       "--"),
    (se_pred, "Proposed CORTEX",      "#d62728",  "-"),
]:
    x = np.sort(vals)
    y = np.arange(1, len(x)+1) / len(x)
    ax.plot(x, y, color=color, linestyle=ls, label=label)

med_ratio = np.median(se_pred) / np.median(se_opt)
ax.text(0.05, 0.55,
        f"Median efficiency:\n{med_ratio*100:.1f}%",
        transform=ax.transAxes,
        bbox=dict(facecolor="white", alpha=0.8, edgecolor="none"))

ax.set(xlabel="Spectral Efficiency (bps/Hz)", ylabel="CDF", xlim=(0, None), ylim=(0, 1.05))
ax.legend(loc="lower right"); ax.grid()
plt.tight_layout()
save_fig("fig_cdf_spectral_efficiency")
plt.show()

In [ ]:
# ── (c) Capacity time-series ──────────────────────────────────────────────────
subset = results.iloc[:min(200, len(results))]

fig, ax = plt.subplots(figsize=(7.16, 3.0))
ax.plot(subset["time"], subset["opt_capacity"], color="k",       alpha=0.4, lw=1,   label="Optimal")
ax.plot(subset["time"], subset["capacity"],     color="#1f77b4", lw=1.5,           label="CORTEX")
ax.set(xlabel="Time (s)", ylabel="Capacity (Gbps)",
       xlim=(subset["time"].iloc[0], subset["time"].iloc[-1]))
ax.legend(ncol=2); ax.grid()
plt.tight_layout()
save_fig("fig_rate_tracking")
plt.show()

print(f"\nAll plots saved to: {PLOTS_DIR}")

## Citation

```bibtex
@article{usta2026cortex,
  author  = {Usta, Mahir Burak and Bafarassat, Milad and Erdem, Mikail and
             Gurbuz, Ozgur and Saeed, Akhtar and Tokgoz, Korkut Kaan and Qaraqe, Khalid},
  title   = {Transformer-Driven Beam Control via Reconfigurable Antenna Arrays
             for Terahertz {UAV} Communications},
  journal = {IEEE Open Journal of the Communications Society},
  year    = {2026},
}
```